In [10]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [11]:
mnist = tf.keras.datasets.mnist
#tune float
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

keras方式训练模型

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [4]:
model.fit(x_train, y_train, epochs=5)

model.evaluate(x_test,  y_test, verbose=2)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 5s 91us/sample - loss: 0.2969 - accuracy: 0.9144
Epoch 2/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.1446 - accuracy: 0.9568
Epoch 3/5
60000/60000 [==============================] - 5s 88us/sample - loss: 0.1075 - accuracy: 0.9679
Epoch 4/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.0877 - accuracy: 0.9735
Epoch 5/5
60000/60000 [==============================] - 5s 85us/sample - loss: 0.0742 - accuracy: 0.9772
10000/10000 - 1s - loss: 0.0799 - accuracy: 0.9757


[0.0799021470952779, 0.9757]

Tensorflow2.x训练方式

In [12]:
# Add a channels dimension
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

#tf.data
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

In [13]:
class MyModel(Model):
    #define model type
    def __init__(self):
        super(MyModel, self).__init__()
        #Sequential
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')
    
    #inputs
    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [14]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function                             #acceleration training 
def train_step(images, labels):
    with tf.GradientTape() as tape:                     #calculate Gradient connect def and var
        predictions = model(images)                     #Tensor("my_model/dense_3/Softmax:0")
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)                      #trans type
    train_accuracy(labels, predictions)

In [15]:
#测试模型
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)                     #trans type
    test_accuracy(labels, predictions)

In [16]:
#训练模型
N_EPOCHS = 5
for epoch in range(1,N_EPOCHS+1):
    # 在下一个epoch开始时，重置评估指标,no necessary
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

    for images, labels in train_ds:
        train_step(images, labels)

    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print (template.format(epoch,
                           train_loss.result(),                              #tensor tune float
                           train_accuracy.result()*100,
                           test_loss.result(),
                           test_accuracy.result()*100))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 1, Loss: 0.13099157810211182, Accuracy: 96.06666564941406, Test Loss: 0.06699678301811218, Test Accuracy: 97.79999542236328
Epoch 2, Loss: 0.04127798601984978, Accuracy: 98.7316665649414, Test Loss: 0.05322837084531784, Test Accuracy: 98.25
Epoch 3, Loss: 0.020942630246281624, Accuracy: 99.32833099365234, Test Loss: 0.05569441616535187, Test Accuracy: 98.4000015258789
Epoch 4, Loss: 0.012188010849058628, Accuracy: 99.58666229248047, Test Loss: 0.05575696378946304, Test Accuracy: 98.33999633789062
